Una versión para los Potenciales separados por bandas.

In [1]:
push!(LOAD_PATH, ".")

4-element Array{String,1}:
 "@"
 "@v#.#"
 "@stdlib"
 "."

In [2]:
using HDF5
using LindenbergOperadores, ParaSets
#importall LindenbergOperadores

┌ Info: Precompiling ParaSets [top-level]
└ @ Base loading.jl:1260


In [3]:
arxnombre="../JLDFiles/Deshinibida/180116s1/180116s1cut_evento_1_bandaslfp.h5"
arx=h5open(arxnombre)
nambandas=names(arx)

7-element Array{String,1}:
 "alpha"
 "beta"
 "delta"
 "gamma"
 "spr"
 "theta"
 "uspr"

In [4]:
nomutiles=["alpha", "beta", "gamma", "spr", "uspr"]

5-element Array{String,1}:
 "alpha"
 "beta"
 "gamma"
 "spr"
 "uspr"

Theta y Delta no tienen nada en el archivo de prueba, 180116s1cut_evento_1

In [5]:
arxbasenombre="../JLDFiles/Deshinibida/180116s1/180116s1cut_evento_1.h5"
arxbase=h5open(arxbasenombre)
nambase=names(arxbase)

7-element Array{String,1}:
 "CMPNeg"
 "CMPos"
 "CSDLindenberg"
 "CanalesSaturados"
 "Canalesrespuesta"
 "LFPTotal"
 "freq"

In [6]:
saturadosarray=read(arxbase["CanalesSaturados"])
respuestasarray=read(arxbase["Canalesrespuesta"]);
frecuencia=read(arxbase["freq"])
close(arxbase)

In [7]:
saturados=arraytoset(saturadosarray)
respuestas=arraytoset(respuestasarray);

In [8]:
lfps=Dict{String, Array}()

Dict{String,Array} with 0 entries

In [9]:
for n in nomutiles
    lfps[n]=read(arx[n])
end
nmax=size(lfps["alpha"],3)

2001

In [10]:
for n in nomutiles
#Poner en cero los canales inservibles
    for m in saturados
        q=m[1]
        p=m[2]
        lfps[n][q,p,:]=zeros(nmax)
    end
end

In [11]:
#Creamos una mancha suave sobre el canal saturado.
for n in nomutiles
    for m in saturados
        q=m[1]
        p=m[2]
        vecinos=vecindad8(m)
        lfps[n][q,p,:]=promediasobreconjunto(vecinos,lfps[n])
    end
end

In [13]:
(mu, nu, lu)=size(lfps["alpha"])

(64, 64, 2001)

In [14]:
csds=Dict{String, Array}()

Dict{String,Array} with 0 entries

In [16]:
for n in nomutiles
    aux1=zeros(mu,nu,lu)
    aux2=zeros(mu,nu,lu)
    #Suavizamos espacialmente con un filtro Gaussiano bidimensional el LFP.
    #Posteriormente sacamos el dCSD.
    for t=1:lu
        aux1[:,:,t]=GaussianSmooth(lfps[n][:,:,t])
        aux2[:,:,t]=DiscreteLaplacian(aux1[:,:,t])
    end
    csds[n]=-aux2;
end

In [25]:
arxout=arxnombre[1:end-6]*"csd.h5"

h5open(arxout, "w") do file
    for n in nomutiles
    write(file, n, csds[n])  # alternatively, say "@write file A"
    end
end

In [26]:
arx=h5open(arxout)
names(arx)

5-element Array{String,1}:
 "alpha"
 "beta"
 "gamma"
 "spr"
 "uspr"

In [28]:
close(arx)

In [29]:
arxout

"../JLDFiles/Deshinibida/180116s1/180116s1cut_evento_1_bandascsd.h5"